In [ ]:
import sys
import os
import pandas as pd
import warnings
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import *
sys.path.append(os.path.abspath("/projects/genomic-ml/da2343/ml_project_1/shared"))
from model_header import *
from constants import *

warnings.filterwarnings('ignore')
np.set_printoptions(threshold=np.inf)

params_df = pd.read_csv("params.csv")
if len(sys.argv) == 2:
    prog_name, task_str = sys.argv
    param_row = int(task_str)
else:
    print("len(sys.argv)=%d so trying first param" % len(sys.argv))
    param_row = 0
    
param_dict = dict(params_df.iloc[param_row, :])
data_set_name = param_dict["Dataset"]
index_of_pred_col = param_dict["Index of Prediction Col"]


dataset_path = dataset_dict[data_set_name]
n_splits = 3

# Import the csv file of the dataset
dataset_pd = pd.read_csv(dataset_path, header=0)
col_names = list(dataset_pd.columns)
# drop only one column per every iteration to form the input matrix
# make the column you removed the output
# print the size of the input matrix
output_vec = dataset_pd.iloc[:, index_of_pred_col].to_frame().to_numpy().ravel()
input_mat = dataset_pd.drop( dataset_pd.columns[index_of_pred_col], axis=1).to_numpy()


# threshold_param_list = np.concatenate(
#     (np.linspace(0, 0.2, 125), np.linspace(0.21, 0.4, 21), np.arange(0.5, 1.01, 0.1)))
# threshold_param_list = np.concatenate((np.linspace(0, 0.4, 5), 
#      np.linspace(0.41, 0.6, 21), np.arange(0.7, 1.01, 0.1)))
threshold_param_list = np.arange(0, 1.01, 0.1)
threshold_param_dict = [{'threshold': [threshold]}
                        for threshold in threshold_param_list]

# alpha_param_list = [10 ** x for x in np.concatenate((np.arange(-7, -2.5, 1), 
#                                        np.linspace(-2.5, -0.01, 50), 
#                                        np.arange(0, 1, 0.5)))]
alpha_param_list = [10 ** x for x in range(-10, 2)]
alpha_param_dict = [{'alpha': [alpha]} for alpha in alpha_param_list]


my_algorithm_list = [
   {
        'learner': GridSearchCV(MyPearsonRegressor(),
                                threshold_param_dict,
                                scoring='neg_mean_squared_error',
                                return_train_score=True),
        'reg_param': 'threshold',
        'name': 'Pearson',
    },
    {
        'learner': GridSearchCV(SpearmanRankRegressor(),
                                threshold_param_dict,
                                scoring='neg_mean_squared_error',
                                return_train_score=True),
        'reg_param': 'threshold',
        'name': 'Spearman',
    },
    {
        'learner': GaussianGraphicalModel(),
        'reg_param': 'alpha',
        'name': 'GGM',
    },
    {
        'learner': GridSearchCV(Lasso(), 
                                alpha_param_dict,
                                scoring='neg_mean_squared_error', 
                                return_train_score=True),
        'reg_param': 'alpha',
        'name': 'LASSO',
    },
]

lasso_coef_df_list = []
pearson_mc_df_list = []
source_target_df_list = []

pearson_source_target_df_list = []
spearman_source_target_df_list = []
ggm_source_target_df_list = []


k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec[index_vec]
        }

    for algorithm_dict in my_algorithm_list:
        my_learner = algorithm_dict["learner"]
        my_reg_param = algorithm_dict["reg_param"]
        my_algo_name = algorithm_dict["name"]

        my_learner.fit(**set_data_dict["train"])
        X_train = set_data_dict["train"]["X"]
        y_train = set_data_dict["train"]["y"]
        X_train_ranked = ss.rankdata(X_train, axis=0)
        y_train_ranked = ss.rankdata(y_train)
        

        if my_algo_name == 'LASSO':
            hyperparam_list = my_learner.cv_results_['params']
            mean_train_score_list = my_learner.cv_results_['mean_train_score'] * -1
            mean_test_score_list = my_learner.cv_results_['mean_test_score'] * -1
            
            lasso_mc_df_list = []
            estimator = Lasso()
            for hyperparam in hyperparam_list:
                estimator.set_params(**hyperparam)
                estimator.fit(**set_data_dict["train"])
                coef_list = estimator.coef_.tolist()
                coef_list.insert(index_of_pred_col, None)
                coef_array = np.array(coef_list)
                score_index = hyperparam_list.index(hyperparam)
                lasso_mc_df_list.append(
                    {
                        'subtrain_score': mean_train_score_list[score_index],
                        'validation_score': mean_test_score_list[score_index],
                        'reg_param': hyperparam['alpha'],
                        'algorithm': my_algo_name,
                        'data_set_name': data_set_name,
                        'fold_id': fold_id,
                        'index_of_pred_col': index_of_pred_col,
                        'coefs': coef_array,
                    }
                ) 
            lasso_coef_df_list.append(pd.DataFrame(lasso_mc_df_list))

final_pearson_corr_df = pd.concat(pearson_mc_df_list)
final_lasso_coef_df = pd.concat(lasso_coef_df_list)
final_pearson_source_target_df = pd.concat(pearson_source_target_df_list)
final_spearman_source_target_df = pd.concat(spearman_source_target_df_list)

if index_of_pred_col == 0:
    final_ggm_source_target_df = pd.concat(ggm_source_target_df_list)
    final_ggm_source_target_df.to_csv(f"ggm_source_target/{param_row}.csv", encoding='utf-8', index=False)

# Save dataframe as a csv to output directory
# print(main_test_df)
# main_test_df.to_csv(f"results/{param_row}.csv", encoding='utf-8', index=False)
# final_lasso_coef_df.to_csv(f"lasso_coef/{param_row}.csv", encoding='utf-8', index=False)
# final_pearson_corr_df.to_csv(f"pearson_corr/{param_row}.csv", encoding='utf-8', index=False)

# final_pearson_source_target_df.to_csv(f"pearson_source_target/{param_row}.csv", encoding='utf-8', index=False)
# final_spearman_source_target_df.to_csv(f"spearman_source_target/{param_row}.csv", encoding='utf-8', index=False)
# print("Done!!")


In [1]:
# Import the modules
import time
import multiprocessing
import threading

global_var = "hello world"
# Define the function to calculate the sum of squares
def sum_squares(n):
    s = 0
    print(global_var)
    for i in range(1, n+1):
        s += i*i
    return s

# Define the number of processes or threads to use
num_workers = 9

# Define the range of numbers to calculate
num_range = [25, 50, 750, 100]

# Define the multiprocessing function
def multiprocessing_func():
    # Create a pool of processes
    pool = multiprocessing.Pool(num_workers)
    # Start the timer
    start_time = time.time()
    # Map the function to the range of numbers
    result = pool.map(sum_squares, num_range)
    # Close the pool
    pool.close()
    # Join the processes
    pool.join()
    # Stop the timer
    end_time = time.time()
    # Print the result and the time taken
    print(f"Multiprocessing result: {result}")
    print(f"Multiprocessing time: {end_time - start_time}")

# Run the multiprocessing function
multiprocessing_func()


hello worldhello worldhello worldhello world



Multiprocessing result: [5525, 42925, 140906375, 338350]
Multiprocessing time: 0.012546062469482422


In [ ]:
def get_model_selection_params(index_of_pred_col):
    input_mat, output_vec = prep_data(dataset_pd, index_of_pred_col)
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec[index_vec]
        }
    my_learner = GridSearchCV(MyPearsonRegressor(),
                        threshold_param_dict,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)
    my_learner.fit(**set_data_dict["train"])
    X_train = set_data_dict["train"]["X"]
    y_train = set_data_dict["train"]["y"]
    params_list = get_corr_hyper_params(X = X_train, 
                                        y = y_train, 
                                        cv_results = my_learner.cv_results_)
    pearson_mc_df = pd.DataFrame(params_list)
    # rename the column name threshold to reg_param
    pearson_mc_df.rename(columns={my_reg_param: 'reg_param'}, inplace=True)
    pearson_mc_df['subtrain_score'] = my_learner.cv_results_['mean_train_score'] * -1
    pearson_mc_df['validation_score'] = my_learner.cv_results_['mean_test_score'] * -1
    pearson_mc_df['index_of_pred_col'] = index_of_pred_col      
    return pearson_mc_df  

num_workers = 9

index_of_pred_col_range = list(range(0, 10))
# Define the multiprocessing function
def multiprocessing_func():
    # Create a pool of processes
    pool = multiprocessing.Pool(num_workers)
    # Map the function to the range of numbers
    result = pool.map(get_model_selection_params, index_of_pred_col_range)
    pool.close()
    pool.join()
    # Print the result and the time taken
    print(f"Multiprocessing result: {result}")

# Run the multiprocessing function
multiprocessing_func()

In [2]:
# Import pandas
import pandas as pd
import numpy as np

my_list = []
coef_list = [1, 2, 3, 4, 5]
coef_array = np.array(coef_list)
my_list.append(
    {
        'coefs': coef_array,
    }
)

df = pd.DataFrame(my_list)
print(df)


             coefs
0  [1, 2, 3, 4, 5]


In [5]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    {'coefs': [np.array([1, 2, 3, 4, 5])],
     'index_of_pred_col': [0]
        }
    )
print(df)


             coefs  index_of_pred_col
0  [1, 2, 3, 4, 5]                  0


In [ ]:
import sys
import os
import pandas as pd
import warnings
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import *
sys.path.append(os.path.abspath("/projects/genomic-ml/da2343/ml_project_1/shared"))
from model_header import *
from constants import *

warnings.filterwarnings('ignore')
np.set_printoptions(threshold=np.inf)

params_df = pd.read_csv("params.csv")
if len(sys.argv) == 2:
    prog_name, task_str = sys.argv
    param_row = int(task_str)
else:
    print("len(sys.argv)=%d so trying first param" % len(sys.argv))
    param_row = 0
    
param_dict = dict(params_df.iloc[param_row, :])
data_set_name = param_dict["Dataset"]
index_of_pred_col = param_dict["Index of Prediction Col"]


dataset_path = dataset_dict[data_set_name]
n_splits = 3

# Import the csv file of the dataset
dataset_pd = pd.read_csv(dataset_path, header=0)
col_names = list(dataset_pd.columns)
# drop only one column per every iteration to form the input matrix
# make the column you removed the output
# print the size of the input matrix
output_vec = dataset_pd.iloc[:, index_of_pred_col].to_frame().to_numpy().ravel()
input_mat = dataset_pd.drop( dataset_pd.columns[index_of_pred_col], axis=1).to_numpy()


threshold_param_list = np.arange(0, 1.01, 0.1)
threshold_param_dict = [{'threshold': [threshold]}
                        for threshold in threshold_param_list]

lasso_coef_df_list = []
pearson_mc_df_list = []
source_target_df_list = []

pearson_source_target_df_list = []
spearman_source_target_df_list = []


my_learner = GridSearchCV(MyPearsonRegressor(),
                        threshold_param_dict,
                        scoring='neg_mean_squared_error',
                        return_train_score=True)
my_reg_param = 'threshold'
my_algo_name = 'Pearson'

k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec[index_vec]
        }

    my_learner.fit(**set_data_dict["train"])
    X_train = set_data_dict["train"]["X"]
    y_train = set_data_dict["train"]["y"]
    params_list = get_corr_hyper_params(X = X_train, 
                                        y = y_train, 
                                        cv_results = my_learner.cv_results_)
    pearson_mc_df = pd.DataFrame(params_list)
    # rename the column name threshold to reg_param
    pearson_mc_df.rename(columns={my_reg_param: 'reg_param'}, inplace=True)
    pearson_mc_df['subtrain_score'] = my_learner.cv_results_['mean_train_score'] * -1
    pearson_mc_df['validation_score'] = my_learner.cv_results_['mean_test_score'] * -1
    pearson_mc_df['index_of_pred_col'] = index_of_pred_col
    pearson_mc_df_list.append(pearson_mc_df)

    # Create the source-target dataframe
    best_reg_param = pearson_mc_df.loc[pearson_mc_df['validation_score'].idxmin(), 'reg_param']
    source_target = get_corr_source_target(
        X = X_train, 
        y = y_train, 
        index = index_of_pred_col,
        threshold = best_reg_param,
    )
    source_target_df = pd.DataFrame(source_target, 
                                    columns=["source", "target", "weight"])
    source_target_df['fold_id'] = fold_id
    pearson_source_target_df_list.append(source_target_df)
    
    
final_pearson_corr_df = pd.concat(pearson_mc_df_list)
final_pearson_source_target_df = pd.concat(pearson_source_target_df_list)


# Save dataframe as a csv to output directory
# print(main_test_df)
# main_test_df.to_csv(f"results/{param_row}.csv", encoding='utf-8', index=False)
# final_lasso_coef_df.to_csv(f"lasso_coef/{param_row}.csv", encoding='utf-8', index=False)
# final_pearson_corr_df.to_csv(f"pearson_corr/{param_row}.csv", encoding='utf-8', index=False)

# final_pearson_source_target_df.to_csv(f"pearson_source_target/{param_row}.csv", encoding='utf-8', index=False)
# final_spearman_source_target_df.to_csv(f"spearman_source_target/{param_row}.csv", encoding='utf-8', index=False)
# print("Done!!")
